In [1]:
import sys
import numpy as np
from ctypes import *
import ctypes
import torch
import math
import random

if "/home/yskim/projects/sparse-bfp" not in sys.path:
    sys.path.append("/home/yskim/projects/sparse-bfp")
from util.reprod_util import set_reproducibility

c_lib = ctypes.CDLL("/home/yskim/projects/sparse-bfp/util/sample/lib_sample.so")

In [ ]:
def split_bit_level(value):
    value_str = f"{value:08b}"
    return [
        int(value_str[0:2], 2),
        int(value_str[2:2+2], 2),
        int(value_str[4:4+2], 2),
        int(value_str[6:6+2], 2),
    ]


def decide_precision(M, group_size):
    value_threshold = group_size * 3 * 0.5
    curr_precision = 2

    results = []
    for i in range(group_size):
        results.append(split_bit_level(M[i]))
    # print(results)

    zero_pass = [False] * group_size

    for curr_level in range(3):
        for i in range(group_size):
            if results[i][curr_level] != 0:
                zero_pass[i] = True
        
        is_pass_zero = True
        for i in range(group_size):
            if zero_pass[i] == False:
                is_pass_zero = False
                break

        if is_pass_zero:
            value_sum = 0
            for i in range(group_size):
                value_sum += results[i][curr_level]
            if value_sum > value_threshold:
                return curr_precision

        curr_precision += 2

    return curr_precision


def change_prec(M, M_compressed, precision, group_size):
    offset = np.uint32(8 - precision)
    x_2 = np.uint32(1 << offset)
    x_1 = np.uint32(0)
    
    for i in range(group_size):
        x = np.uint32(int(f"{M[i]:08b}", 2))
        x <<= (np.uint32(32) - offset)
        x >>= (np.uint32(32) - offset)

        prob = random.uniform(0, 1)
        prob_to_left = (x_2 - x) / (x_2 - x_1)

        value = int(f"{M[i]:08b}"[0:precision], 2) + (1 if prob >= prob_to_left else 0)

        M_compressed[i] = value
        # print(f"{M[i]:08b} {value:08b}")


class BfpWrapper:
    def __init__(
        self, 
        vec: torch.Tensor,
        vec_size,
        bfp_M_bit,
        group_size):
        super().__init__()

        self.vec = vec
        self.vec_size = vec_size
        self.bfp_M_bit = bfp_M_bit
        self.group_size = group_size

        self.S = torch.zeros(vec_size, dtype=torch.int32)
        self.E = torch.zeros(vec_size, dtype=torch.int32)
        self.M = torch.zeros(vec_size, dtype=torch.int64)
        self.M_compressed = torch.zeros(vec_size, dtype=torch.int64)

        self.bfp_M_bits = torch.zeros(vec_size, dtype=torch.int32)
        self.bfp_M_bits[:] = bfp_M_bit
        # print(self.bfp_M_bits)

        self.bit_levels = torch.zeros(vec_size, dtype=torch.int32)
        

    def convert(self):
        c_lib.convert(
        c_void_p(self.vec.data_ptr()),
        c_void_p(self.S.data_ptr()),
        c_void_p(self.E.data_ptr()),
        c_void_p(self.M.data_ptr()),
        c_int32(self.bfp_M_bit),
        c_int32(self.vec_size),
        c_int32(self.group_size))

       
    def compress(self):
        c_lib.compress(
        c_void_p(self.vec.data_ptr()),
        c_void_p(self.S.data_ptr()),
        c_void_p(self.E.data_ptr()),
        c_void_p(self.M.data_ptr()),
        c_void_p(self.bfp_M_bits.data_ptr()),
        c_int32(self.vec_size),
        c_int32(self.group_size))

        return
        steps = int(math.floor(self.vec_size / self.group_size))
        tail = self.vec_size % self.group_size

        for i in range(steps):
            start = self.group_size * i
            end = start + self.group_size
            precision = decide_precision(self.M[start:end], self.group_size)
            # print(precision)
            self.bit_levels[start:end] = precision
            change_prec(self.M[start:end], self.M_compressed[start:end], precision, self.group_size)



        if tail > 0:
            start = self.group_size * steps
            end = start + tail
            precision = decide_precision(self.M[start:end], tail)
            self.bit_levels[start:end] = precision
            change_prec(self.M[start:end], self.M_compressed[start:end], precision, tail)

        # print(self.bit_levels)

        # for i in range(self.vec_size):
        #     print(f"{self.M[i].item():08b} {self.M_compressed[i].item():08b}")

def dp(
    lhs_S,
    lhs_E,
    lhs_M,
    lhs_M_bits,
    
    rhs_S,
    rhs_E,
    rhs_M,
    rhs_M_bits,
    
    vec_size,
    group_size):
    steps = int(math.floor(vec_size / group_size))
    tail = vec_size % group_size

    result = 0.

    # bits = []

    for step in range(steps):
        start = group_size * step
        mantissa = 0

        # lhs_bit = lhs_M_bits[start]
        # rhs_bit = rhs_M_bits[start]

        # if lhs_bit != rhs_bit:
        #     if lhs_bit > rhs_bit:
        #         lhs_offset = lhs_bit - rhs_bit
        #         rhs_offset = 0
        #         lhs_bit = rhs_bit

        #     else:
        #         lhs_offset = 0
        #         rhs_offset = rhs_bit - lhs_bit
        #         rhs_bit = lhs_bit
        # else:
        #     lhs_offset = 0
        #     rhs_offset = 0

        for i in range(start, start + group_size):
            mantissa += (lhs_S[i] * lhs_M[i]) * (rhs_S[i] * rhs_M[i])
            # mantissa += (lhs_S[i] * (lhs_M[i] >> lhs_offset)) * (rhs_S[i] * (rhs_M[i] >> rhs_offset))

        sign = 1.
        if mantissa < 0:
            sign = -1.
            mantissa *= -1

        double_mantissa = mantissa / pow(2., (lhs_M_bits[start] - 1) + (rhs_M_bits[start] - 1))
        # double_mantissa = mantissa / pow(2., (lhs_bit - 1) + (rhs_bit - 1))

        # bits.append(lhs_bit)
        # bits.append(rhs_bit)

        result += sign * pow(2., lhs_E[start] + rhs_E[start]) * double_mantissa

    # print(f"mean bits: {np.mean(bits)}")

    if tail > 0:
        raise ValueError("not supported: implement it!")
        start = group_size * steps
        mantissa = 0

        for i in range(start, start + tail):
            mantissa += (lhs_S[i] * lhs_M[i]) * (rhs_S[i] * rhs_M[i])

        sign = 1.
        if mantissa < 0:
            sign = -1.
            mantissa *= -1

        double_mantissa = mantissa / pow(2., (lhs_M_bits[start] - 1) + (rhs_M_bits[start] - 1))

        result += sign * pow(2., lhs_E[start] + rhs_E[start]) * double_mantissa

    return result

    # return sign * pow(2., lhs_E[0] + rhs_E[0]) * double_mantissa

def load_inout(path_root, phase, layer_index, n_index, k_cnt, m_index):
    path = f"{path_root}/{phase}-{layer_index}-fp"

    lhs = np.load(f"{path}-lhs.npy", allow_pickle=False)
    rhs = np.load(f"{path}-rhs.npy", allow_pickle=False)
    output = np.matmul(lhs, rhs)
    print(f"lhs: {lhs.shape} X rhs: {rhs.shape} = output: {output.shape}")
    
    lhs = lhs[n_index,0:k_cnt]
    rhs = rhs[0:k_cnt,m_index]
    output = np.dot(lhs, rhs)
    print(f"lhs: {lhs.shape} X rhs: {rhs.shape} = output: {output.shape}")

    return lhs, rhs, output

In [ ]:
# set_reproducibility(1234)

vec_size = 128
bfp_M_bit = 8
group_size = 16

# lhs_vec = torch.abs(torch.randn(vec_size, dtype=torch.float32))
# rhs_vec = torch.abs(torch.randn(vec_size, dtype=torch.float32))
# lhs_vec = torch.randn(vec_size, dtype=torch.float32)
# rhs_vec = torch.randn(vec_size, dtype=torch.float32)

log_path = "/home/yskim/projects/sparse-bfp/resnet50-inout-log/epoch0-lr0.001"
lhs, rhs, output = load_inout(log_path, "fwd", 49, 32, vec_size, 34)

lhs_vec = torch.tensor(lhs)
rhs_vec = torch.tensor(rhs)

lhs_wrapper = BfpWrapper(lhs_vec, vec_size, bfp_M_bit, group_size)
lhs_wrapper.convert()
# lhs_wrapper.compress()

lhs_wrapper_6bit = BfpWrapper(lhs_vec, vec_size, 6, group_size)
lhs_wrapper_6bit.convert()

lhs_wrapper_4bit = BfpWrapper(lhs_vec, vec_size, 4, group_size)
lhs_wrapper_4bit.convert()

lhs_wrapper_2bit = BfpWrapper(lhs_vec, vec_size, 2, group_size)
lhs_wrapper_2bit.convert()

lhs_wrapper_flex = BfpWrapper(lhs_vec, vec_size, bfp_M_bit, group_size)
lhs_wrapper_flex.compress()

rhs_wrapper = BfpWrapper(rhs_vec, vec_size, bfp_M_bit, group_size)
rhs_wrapper.convert()
# rhs_wrapper.compress()

rhs_wrapper_6bit = BfpWrapper(rhs_vec, vec_size, 6, group_size)
rhs_wrapper_6bit.convert()

rhs_wrapper_4bit = BfpWrapper(rhs_vec, vec_size, 4, group_size)
rhs_wrapper_4bit.convert()

rhs_wrapper_2bit = BfpWrapper(rhs_vec, vec_size, 2, group_size)
rhs_wrapper_2bit.convert()

rhs_wrapper_flex = BfpWrapper(rhs_vec, vec_size, bfp_M_bit, group_size)
rhs_wrapper_flex.compress()
# print(rhs_wrapper_flex.M.numpy())
# print(rhs_wrapper_flex.bfp_M_bits)


# print(torch.dot(lhs_vec, rhs_vec))

bfp_8bit = dp(
    lhs_wrapper.S,
    lhs_wrapper.E,
    lhs_wrapper.M,
    lhs_wrapper.bfp_M_bits,
    
    rhs_wrapper.S,
    rhs_wrapper.E,
    rhs_wrapper.M,
    rhs_wrapper.bfp_M_bits,

    vec_size,
    group_size
)

bfp_6bit = dp(
    lhs_wrapper_6bit.S,
    lhs_wrapper_6bit.E,
    lhs_wrapper_6bit.M,
    lhs_wrapper_6bit.bfp_M_bits,

    rhs_wrapper_6bit.S,
    rhs_wrapper_6bit.E,
    rhs_wrapper_6bit.M,
    rhs_wrapper_6bit.bfp_M_bits,

    vec_size,
    group_size
)

bfp_4bit = dp(
    lhs_wrapper_4bit.S,
    lhs_wrapper_4bit.E,
    lhs_wrapper_4bit.M,
    lhs_wrapper_4bit.bfp_M_bits,
    
    rhs_wrapper_4bit.S,
    rhs_wrapper_4bit.E,
    rhs_wrapper_4bit.M,
    rhs_wrapper_4bit.bfp_M_bits,

    vec_size,
    group_size
)

bfp_2bit = dp(
    lhs_wrapper_2bit.S,
    lhs_wrapper_2bit.E,
    lhs_wrapper_2bit.M,
    lhs_wrapper_2bit.bfp_M_bits,

    rhs_wrapper_2bit.S,
    rhs_wrapper_2bit.E,
    rhs_wrapper_2bit.M,
    rhs_wrapper_2bit.bfp_M_bits,

    vec_size,
    group_size
)

bfp_flex = dp(
    lhs_wrapper_flex.S,
    lhs_wrapper_flex.E,
    lhs_wrapper_flex.M,
    lhs_wrapper_flex.bfp_M_bits,
    
    rhs_wrapper_flex.S,
    rhs_wrapper_flex.E,
    rhs_wrapper_flex.M,
    rhs_wrapper_flex.bfp_M_bits,

    vec_size,
    group_size
)

fp32 = torch.dot(lhs_vec, rhs_vec).item()

# print(f"8bit: {lhs_wrapper.S.numpy()} {rhs_wrapper.S.numpy()}")
# print(f"6bit: {lhs_wrapper_6bit.S.numpy()} {rhs_wrapper_6bit.S.numpy()}")
# print()

# print(f"8bit: {lhs_wrapper.E.numpy()} {rhs_wrapper.E.numpy()}")
# print(f"6bit: {lhs_wrapper_6bit.E.numpy()} {rhs_wrapper_6bit.E.numpy()}")
# print()

# print(f"8bit: {[f'{e:08b}'for e in lhs_wrapper.M.numpy()]} {[f'{e:08b}'for e in rhs_wrapper.M.numpy()]}")
# print(f"6bit: {[f'{e:08b}'for e in lhs_wrapper_6bit.M.numpy()]} {[f'{e:08b}'for e in rhs_wrapper_6bit.M.numpy()]}")

print(f"fp32:  {fp32:.10f}")
print(f"8-bit: {bfp_8bit:.10f} (diff: {abs(fp32 - bfp_8bit):.10f}), 8-bit")
print(f"6-bit: {bfp_6bit:.10f} (diff: {abs(fp32 - bfp_6bit):.10f}), 6-bit")
print(f"4-bit: {bfp_4bit:.10f} (diff: {abs(fp32 - bfp_4bit):.10f}), 4-bit")
print(f"2-bit: {bfp_2bit:.10f} (diff: {abs(fp32 - bfp_2bit):.10f}), 2-bit")
print(f"flex : {bfp_flex:.10f} (diff: {abs(fp32 - bfp_flex):.10f}), {np.mean(lhs_wrapper_flex.bfp_M_bits.numpy()+rhs_wrapper_flex.bfp_M_bits.numpy()) / 2:.2f}-bit")

e = [lhs_wrapper_flex.E[i * group_size].item() for i in range(vec_size // group_size)]
print(f"{e}")
e = [rhs_wrapper_flex.E[i * group_size].item() for i in range(vec_size // group_size)]
print(f"{e}")

# print(lhs_wrapper.E)

# mean bits: 4.75
# fp32:  4.893858
# 8-bit: 4.830475 (diff: 0.063384), 8-bit
# 6-bit: 4.630615 (diff: 0.263243), 6-bit
# 4-bit: 3.300781 (diff: 1.593077), 4-bit
# flex : 4.505280 (diff: 0.388579), 4.75-bit

# fp32:  4.893858
# 8-bit: 4.830475 (diff: 0.063384), 8-bit
# 6-bit: 4.630615 (diff: 0.263243), 6-bit
# 4-bit: 3.300781 (diff: 1.593077), 4-bit
# flex : 4.536407 (diff: 0.357451), 5.91-bit


# mean bits: 8.0
# mean bits: 6.0
# mean bits: 4.0
# mean bits: 6.25
# fp32:  7.195905
# 8-bit: 7.186523 (diff: 0.009381), 8-bit
# 6-bit: 7.197266 (diff: 0.001361), 6-bit
# 4-bit: 6.468750 (diff: 0.727155), 4-bit
# flex : 6.922668 (diff: 0.273236), 6.25-bit

# fp32:  7.195905
# 8-bit: 7.186523 (diff: 0.009381), 8-bit
# 6-bit: 7.197266 (diff: 0.001361), 6-bit
# 4-bit: 6.468750 (diff: 0.727155), 4-bit
# flex : 7.526184 (diff: 0.330279), 7.06-bit

# mean bits: 8.0
# mean bits: 6.0
# mean bits: 4.0
# mean bits: 6.875
# fp32:  -3.044195
# 8-bit: -3.088562 (diff: 0.044367), 8-bit
# 6-bit: -2.643555 (diff: 0.400640), 6-bit
# 4-bit: -1.046875 (diff: 1.997320), 4-bit
# flex : -3.132263 (diff: 0.088068), 7.25-bit

In [ ]:
print("8-bit")
print([f"{e:08b}" for e in lhs_wrapper.M.numpy()])
print([f"{e:08b}" for e in rhs_wrapper.M.numpy()])
print()

print("6-bit")
print([f"{e:08b}" for e in lhs_wrapper_6bit.M.numpy()])
print([f"{e:08b}" for e in rhs_wrapper_6bit.M.numpy()])
print()

print("4-bit")
print([f"{e:08b}" for e in lhs_wrapper_4bit.M.numpy()])
print([f"{e:08b}" for e in rhs_wrapper_4bit.M.numpy()])
print()

print("2-bit")
print([f"{e:08b}" for e in lhs_wrapper_2bit.M.numpy()])
print([f"{e:08b}" for e in rhs_wrapper_2bit.M.numpy()])
print()

print("flex")
print([f"{e:08b}" for e in lhs_wrapper_flex.M.numpy()])
print(lhs_wrapper_flex.bfp_M_bits)
print([f"{e:08b}" for e in rhs_wrapper_flex.M.numpy()])
print(rhs_wrapper_flex.bfp_M_bits)
print()

In [ ]:
print(lhs_wrapper.S.numpy())
print(lhs_wrapper.E.numpy())
print(rhs_wrapper.S.numpy())
print(rhs_wrapper.E.numpy())
print()
print(lhs_wrapper_4bit.S.numpy())
print(lhs_wrapper_4bit.E.numpy())
print(rhs_wrapper_4bit.S.numpy())
print(rhs_wrapper_4bit.E.numpy())

In [ ]:
lhs_8bit = ['01011000', '11000001', '01011000', '00101011', '01010111', '10011011', '00010001', '00101101']
lhs_8bit = [int(e, 2) for e in lhs_8bit]
lhs_8bit_M_bits = [8] * 2
rhs_8bit = ['00111100', '00101110', '10111110', '01001111', '01110110', '00111011', '01011011', '10100011']
rhs_8bit = [int(e, 2) for e in rhs_8bit]
rhs_8bit_M_bits = [8] * 2


lhs_6bit = ['00010110', '00110000', '00010110', '00001011', '00010110', '00100111', '00000101', '00001100']
lhs_6bit = [int(e, 2) for e in lhs_6bit]
lhs_6bit_M_bits = [6] * 2
rhs_6bit = ['00001111', '00001100', '00110000', '00010100', '00011101', '00001110', '00010110', '00101001']
rhs_6bit = [int(e, 2) for e in rhs_6bit]
rhs_6bit_M_bits = [6] * 2


lhs_flex = ['01011000', '11000001', '01011000', '00101011', '01011000', '10011100', '00010010', '00101101']
lhs_flex = [int(e, 2) for e in lhs_flex]
lhs_flex_M_bits = [8,8]
rhs_flex = ['00000100', '00000011', '00001100', '00000101', '00000111', '00000100', '00000110', '00001010']
rhs_flex = [int(e, 2) for e in rhs_flex]
rhs_flex_M_bits = [4,4]

# ['00000101', '00001100', '00000101', '00000011', '00000110', '00001010', '00000001', '00000011']
# ['00000101', '00001100', '00000101', '00000010', '00000101', '00001001', '00000001', '00000010'] nSR
lhs_md = ['00000101', '00001100', '00000101', '00000011', '00000110', '00001010', '00000001', '00000011']
lhs_md = [int(e, 2) for e in lhs_md]
lhs_md_M_bits = [4,4]
rhs_md = ['00000011', '00000011', '00001100', '00000101', '00001000', '00000011', '00000101', '00001010']
rhs_md = [int(e, 2) for e in rhs_md]
rhs_md_M_bits = [4,4]
# ['00000100', '00000011', '00001100', '00000101', '00000111', '00000100', '00000110', '00001010']

lhs_E = [1,1]
rhs_E = [-1,1]

def dp(lhs_Es, lhs_Ms, lhs_bits, rhs_Es, rhs_Ms, rhs_bits):
    result = 0.
    for i in range(2):
        lhs_bit = lhs_bits[i]
        lhs_E = lhs_Es[i]

        rhs_bit = rhs_bits[i]
        rhs_E = rhs_Es[i]

        start = i * 4
        lhs_M = lhs_Ms[start:start+4]
        rhs_M = rhs_Ms[start:start+4]

        result += pow(2., lhs_E + rhs_E) * (np.dot(lhs_M, rhs_M) / pow(2., lhs_bit + rhs_bit - 2))
        # result += (np.dot(lhs_M, rhs_M) / pow(2., lhs_bit + rhs_bit - 2))
    return result

def dp_man(lhs_Es, lhs_Ms, lhs_bits, rhs_Es, rhs_Ms, rhs_bits):
    result = 0.
    for i in range(2):
        lhs_bit = lhs_bits[i]
        lhs_E = lhs_Es[i]

        rhs_bit = rhs_bits[i]
        rhs_E = rhs_Es[i]

        start = i * 4
        lhs_M = lhs_Ms[start:start+4]
        rhs_M = rhs_Ms[start:start+4]

        # result += pow(2., lhs_E + rhs_E) * (np.dot(lhs_M, rhs_M) / pow(2., lhs_bit + rhs_bit - 2))
        result += (np.dot(lhs_M, rhs_M) / pow(2., lhs_bit + rhs_bit - 2))
    return result

def dp_int(lhs_Es, lhs_Ms, lhs_bits, rhs_Es, rhs_Ms, rhs_bits):
    result = 0
    for i in range(2):
        lhs_bit = lhs_bits[i]
        lhs_E = lhs_Es[i]

        rhs_bit = rhs_bits[i]
        rhs_E = rhs_Es[i]

        start = i * 4
        lhs_M = lhs_Ms[start:start+4]
        rhs_M = rhs_Ms[start:start+4]

        # result += pow(2., lhs_E + rhs_E) * (np.dot(lhs_M, rhs_M) / pow(2., lhs_bit + rhs_bit - 2))
        result += int(np.dot(lhs_M, rhs_M))
    return result
# val = 9.786264
res_8bit = dp(lhs_E, lhs_8bit, lhs_8bit_M_bits, rhs_E, rhs_8bit, rhs_8bit_M_bits)
res_6bit = dp(lhs_E, lhs_6bit, lhs_6bit_M_bits, rhs_E, rhs_6bit, rhs_6bit_M_bits)
res_flex = dp(lhs_E, lhs_flex, lhs_flex_M_bits, rhs_E, rhs_flex, rhs_flex_M_bits)
res_md = dp(lhs_E, lhs_md, lhs_md_M_bits, rhs_E, rhs_md, rhs_md_M_bits)

res_8bit_man = dp_man(lhs_E, lhs_8bit, lhs_8bit_M_bits, rhs_E, rhs_8bit, rhs_8bit_M_bits)
res_6bit_man = dp_man(lhs_E, lhs_6bit, lhs_6bit_M_bits, rhs_E, rhs_6bit, rhs_6bit_M_bits)
res_flex_man = dp_man(lhs_E, lhs_flex, lhs_flex_M_bits, rhs_E, rhs_flex, rhs_flex_M_bits)
res_md_man = dp_man(lhs_E, lhs_md, lhs_md_M_bits, rhs_E, rhs_md, rhs_md_M_bits)

res_8bit_int = dp_int(lhs_E, lhs_8bit, lhs_8bit_M_bits, rhs_E, rhs_8bit, rhs_8bit_M_bits)
res_6bit_int = dp_int(lhs_E, lhs_6bit, lhs_6bit_M_bits, rhs_E, rhs_6bit, rhs_6bit_M_bits)
res_flex_int = dp_int(lhs_E, lhs_flex, lhs_flex_M_bits, rhs_E, rhs_flex, rhs_flex_M_bits)
res_md_int = dp_int(lhs_E, lhs_md, lhs_md_M_bits, rhs_E, rhs_md, rhs_md_M_bits)

print(np.mean(lhs_flex_M_bits))
print(np.mean(rhs_flex_M_bits))
print()
print(f"{res_8bit}")
print(f"{res_6bit}")
print(f"{res_flex}")
print(f"{res_md}")
print()
print(f"{res_8bit_man}")
print(f"{res_6bit_man}")
print(f"{res_flex_man}")
print(f"{res_md_man}")
print()
print(f"{res_8bit_int:032b}")
print(f"{res_6bit_int:032b}")
print(f"{res_flex_int:032b}")
print(f"{res_md_int:032b}")

# 000000000000000011.11010001101000 8 8 
# 0000000000000000000011.1110000000 6 6 
# 0000000000000000000011.1110100000 8 4
# 00000000000000000000000011.101111 4 4

# 11.11010001101000
# 11.1110000000
# 11.1110100000
# 11.101111

In [ ]:
lhs_6bit = ['00001100', '00000011', '00001001', '00101000']
lhs_6bit = [int(e, 2) for e in lhs_6bit]
lhs_6bit_M_bits = [6]
rhs_6bit = ['00110110', '00011011', '00101110', '00010001']
rhs_6bit = [int(e, 2) for e in rhs_6bit]
rhs_6bit_M_bits = [6]

lhs_2bit = ['00000000', '00000000', '00000001', '00000011']
lhs_2bit = [int(e, 2) for e in lhs_2bit]
lhs_2bit_M_bits = [2]
rhs_2bit = ['00000100', '00000001', '00000011', '00000001']
rhs_2bit = [int(e, 2) for e in rhs_2bit]
rhs_2bit_M_bits = [2]


lhs_flex = ['00101111', '00001011', '00100011', '10100001']
lhs_flex = [int(e, 2) for e in lhs_flex]
lhs_flex_M_bits = [8]
rhs_flex = ['00000011', '00000010', '00000010', '00000001']
rhs_flex = [int(e, 2) for e in rhs_flex]
rhs_flex_M_bits = [2]

lhs_E = [1]
rhs_E = [0]

def dp(lhs_Es, lhs_Ms, lhs_bits, rhs_Es, rhs_Ms, rhs_bits):
    result = 0.
    for i in range(1):
        lhs_bit = lhs_bits[i]
        lhs_E = lhs_Es[i]

        rhs_bit = rhs_bits[i]
        rhs_E = rhs_Es[i]

        start = i * 4
        lhs_M = lhs_Ms[start:start+4]
        rhs_M = rhs_Ms[start:start+4]

        # print((np.dot(lhs_M, rhs_M) / pow(2., lhs_bit + rhs_bit - 2)))

        result += pow(2., lhs_E + rhs_E) * (np.dot(lhs_M, rhs_M) / pow(2., lhs_bit + rhs_bit - 2))
    return result
# val = 9.786264
res_6bit = dp(lhs_E, lhs_6bit, lhs_6bit_M_bits, rhs_E, rhs_6bit, rhs_6bit_M_bits)
res_2bit = dp(lhs_E, lhs_2bit, lhs_2bit_M_bits, rhs_E, rhs_2bit, rhs_2bit_M_bits)
res_flex = dp(lhs_E, lhs_flex, lhs_flex_M_bits, rhs_E, rhs_flex, rhs_flex_M_bits)

print(res_6bit)
print(res_2bit)
print(res_flex)

# print(np.abs(res_6bit - val))
# print(np.abs(res_flex - val))

In [ ]:
# lhs_8bit = [
#     int('10100010', 2),
#     int('01011100', 2), 
#     int('11100101', 2), 
#     int('10000110', 2)
# ]
# rhs_8bit = [
#     int('00110010', 2), 
#     int('10111100', 2), 
#     int('01001100', 2), 
#     int('01011101', 2)
# ]
# print(f"{np.dot(lhs_8bit, rhs_8bit) / pow(2., 16 - 2)}")

lhs_6bit = [
    int('00010011', 2), 
    int('00101110', 2), 
    int('00100111', 2), 
    int('00001001', 2)

    # int('00011011', 2),
    # int('00110011', 2), 
    # int('00100010', 2), 
    # int('00111000', 2),
    # int('00010010', 2), 
    # int('00001100', 2), 
    # int('00101010', 2), 
    # int('00100011', 2), 
    # int('00000101', 2), 
    # int('00000110', 2), 
    # int('00100011', 2), 
    # int('00001110', 2), 
    # int('00010011', 2), 
    # int('00101110', 2), 
    # int('00100111', 2), 
    # int('00001001', 2)
]
rhs_6bit = [
    int('00110111', 2), 
    int('00011100', 2), 
    int('00011010', 2), 
    int('00101110', 2)

    # int('00000001', 2), 
    # int('00000100', 2), 
    # int('00100100', 2), 
    # int('00001100', 2),
    # int('00001001', 2),
    # int('00011001', 2), 
    # int('00000100', 2), 
    # int('00100101', 2), 
    # int('00010000', 2), 
    # int('00101000', 2), 
    # int('00010001', 2), 
    # int('00000110', 2), 
    # int('00110111', 2), 
    # int('00011100', 2), 
    # int('00011010', 2), 
    # int('00101110', 2)
]
print(f"{np.dot(lhs_6bit, rhs_6bit) / pow(2., 2 * (6 - 1))}")

# lhs_2bit = [
#     int('00000010', 2),
#     int('00000010', 2), 
#     int('00000011', 2), 
#     int('00000011', 2)
# ]
# rhs_2bit = [
#     int('00000001', 2), 
#     int('00000011', 2), 
#     int('00000001', 2), 
#     int('00000001', 2)
# ]
# print(f"{np.dot(lhs_2bit, rhs_2bit) / pow(2., 4 - 2)}")


lhs_flex = [
    int('00010011', 2), 
    int('00101110', 2), 
    int('00101000', 2), 
    int('00001001', 2)
]
rhs_flex = [
    int('00000011', 2),
    int('00000010', 2), 
    int('00000010', 2), 
    int('00000011', 2)
]
print(f"{np.dot(lhs_flex, rhs_flex) / pow(2., (2 + 6) - 2)}")

In [ ]:
# 8bit: [1 1 1 1] [1 1 1 1]
# 6bit: [1 1 1 1] [1 1 1 1]

# 8bit: [1 1 1 1] [0 0 0 0]
# 6bit: [1 1 1 1] [0 0 0 0]

lhs_M_8_bits = [
    int('01000000', 2), 
    int('00110000', 2), 
    int('00110000', 2), 
    int('10001101', 2)
] 

rhs_M_8_bits = [
    int('00011111', 2),
    int('01000000', 2), 
    int('01010101', 2), 
    int('11011100', 2)
]

lhs_M_6_bits = [
    int('00001111', 2), 
    int('00001100', 2), 
    int('00001100', 2), 
    int('00100100', 2)
] 

rhs_M_6_bits = [
    int('00000111', 2), 
    int('00010000', 2), 
    int('00010101', 2), 
    int('00110111', 2)
]

result_8_bit = pow(2., 1) * (np.dot(lhs_M_8_bits, rhs_M_8_bits) / pow(2., 16 - 2))
result_6_bit = pow(2., 1) * (np.dot(lhs_M_6_bits, rhs_M_6_bits) / pow(2., 12 - 2))

print(result_8_bit)
print(result_6_bit)